# Задание 1.

In [1]:
from string import punctuation
import numpy as np
from nltk.tokenize import sent_tokenize
from collections import Counter

In [2]:
dostoevsky = open('besy_dostoevsky.short').read()

In [3]:
def normalize(text):
    normalized_text = [
        word.strip(punctuation) for word in text.lower().split()
    ]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text

In [4]:
def ngrammer(tokens, n):
    ngrams = []
    for i in range(0, len(tokens) - n+1):
        ngrams.append(' '.join(tokens[i: i+n]))
    return ngrams

In [5]:
sentences_dostoevsky = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dostoevsky)]

In [6]:
unigrams_dostoevsky = Counter()
bigrams_dostoevsky = Counter()
trigrams_dostoevsky = Counter()

for sentence in sentences_dostoevsky:
    unigrams_dostoevsky.update(sentence)
    bigrams_dostoevsky.update(ngrammer(sentence, 2))
    trigrams_dostoevsky.update(ngrammer(sentence, 3))

In [7]:
trigrams_dostoevsky.most_common(10)

[('<start> — я', 133),
 ('<start> — да', 77),
 ('<start> — а', 69),
 ('<start> — вы', 66),
 ('<start> варвара петровна', 53),
 ('<start> — это', 53),
 ('<start> — не', 52),
 ('<start> николай всеволодович', 50),
 ('<start> степан трофимович', 47),
 ('по крайней мере', 42)]

In [8]:
bigrams_dostoevsky.most_common(10)

[('<start> —', 1930),
 ('<start> я', 283),
 ('варвара петровна', 211),
 ('николай всеволодович', 198),
 ('<start> он', 187),
 ('— я', 184),
 ('и не', 182),
 ('<start> но', 181),
 ('степан трофимович', 180),
 ('что я', 147)]

In [9]:
matrix_dostoevsky = np.zeros((
    len(bigrams_dostoevsky),
    len(unigrams_dostoevsky),
))

In [10]:
id2word_dostoevsky = list(unigrams_dostoevsky)
id2word_bigrams_dostoevsky = list(bigrams_dostoevsky)

word2id_dostoevsky = {word: i for i, word in enumerate(id2word_dostoevsky)}
word2id_bigrams_dostoevsky = {word: i for i, word in enumerate(id2word_bigrams_dostoevsky)}

In [11]:
for ngram in trigrams_dostoevsky:
    word1, word2, word3 = ngram.split()
    bigram = '{} {}'.format(word1, word2)
    matrix_dostoevsky[
        word2id_bigrams_dostoevsky[bigram]
    ][
        word2id_dostoevsky[word3]
    ] = (
        trigrams_dostoevsky[ngram] / bigrams_dostoevsky[bigram]
    )

### ремарка
Стоит сказать, что можно было придумать что-то умнее с тегом < start >, но не очень понятно что.
Я попытался заставить выбирать какую-то другую биграмму, если в ней был этот тег. Однако, в таком случае, модель продолжает пихать мне эту же самую биграмму.

In [79]:
def generate(matrix, id2word, word2id, n=100):
    starts = [bigr for bigr in list(word2id) if bigr.startswith('<start>')]
    start = np.random.choice(starts)
    text = []
    current_idx = word2id[start]
    for i in range(n):
        if sum(matrix[current_idx]):
            chosen =\
                np.random.choice(
                    matrix.shape[1],
                    p=matrix[current_idx],
                )
            text.append(id2word[chosen])
        if id2word[chosen].endswith('<end>'):
            chosen = word2id[np.random.choice(starts)]
        current_idx = chosen
    
    return ' '.join(text)

In [81]:
print(
    generate(
        matrix_dostoevsky,
        id2word_bigrams_dostoevsky,
        word2id_bigrams_dostoevsky,
    ).replace(' <end>', '\n').replace(' <start> ', '\n')
)

родного сиволапого
так пришедши к рассказали в и потонуло
он как от многоуважаемой варваре припомню пропел подробностей из в здравом жители смотреть крутизны в и бросилось стадо свиней и они просили его чтобы позволил им войти в них уничтожении наследства постороннему лицу степане трофимовиче трофимовича верховенского
видевшие и пришедши и рассказали в них которым очень конец его для самолюбия боже тем которую я прочла бессмертную фактов явились и они многочтимом степане степана трофимовича
он потом постепенно в них нет
 лучше сказать трофимович постоянно по неумению
часть эту картинку в них которым очень конец его для самолюбия боже тем которую я прочла бессмертную фактов явились и они многочтимом степане степана трофимовича послужат лишь недавних и им как к иисусу в городе вечера стояли не омраченной в петербурге привычки или степан трофимович страшное будет
он луки
 степане трофимовиче трофимовича верховенского
видевшие и пришедши и рассказали в них о талантливом и они многочтимом с

# Задание 2.



In [171]:
from nltk import sent_tokenize
from gensim.models.phrases import Phrases

In [84]:
text = open('anna_karenina.txt').read()

In [118]:
sents = sent_tokenize(text)
sents[100:110]

['И вот возьми телеграмму, передай, что они скажут.',
 '«Попробовать хотите», – понял Матвей, но он сказал только:\n\n– Слушаю-с.',
 'Степан Аркадьич уже был умыт и расчесан и сбирался одеваться, когда Матвей, медленно ступая поскрипывающими сапогами, с телеграммой в руке, вернулся в комнату.',
 'Цирюльника уже не было.',
 '– Дарья Александровна приказали доложить, что они уезжают.',
 'Пускай делают, как им, вам то есть, угодно, – сказал он, смеясь только глазами, и, положив руки в карманы и склонив голову набок, уставился на барина.',
 'Степан Аркадьич помолчал.',
 'Потом добрая и несколько жалкая улыбка показалась на его красивом лице.',
 '– А?',
 'Матвей?']

In [254]:
symbols = [list(sent) for sent in sents]

In [255]:
phrases1 = Phrases(symbols, scoring='npmi', threshold=-1)

In [256]:
phrases2 = Phrases(phrases1[symbols], scoring='npmi', threshold=-1)

In [257]:
phrases3 = Phrases(phrases2[phrases1[symbols]], scoring='npmi', threshold=-1)

In [258]:
phrases4 = Phrases(phrases3[phrases2[phrases1[symbols]]], scoring='npmi', threshold=-1)

In [259]:
phrases5 = Phrases(phrases4[phrases3[phrases2[phrases1[symbols]]]], scoring='npmi', threshold=-1)

In [260]:
phrases = Phrases(phrases5[phrases4[phrases3[phrases2[phrases1[symbols]]]]], scoring='npmi', threshold=-1)

In [261]:
cmn = Counter(phrases.vocab).most_common(15)

In [262]:
for t in cmn:
    print(t[0].decode('utf-8'))

.
?
,_ _и_ 
е_г_о_ 
 _н_е_ 
е_.
!
а_.
 _б_ы_л
 _э_т_о
 _н_а_ 
 _е_г_о
 _о_н_ 
О_н_а_ 
 _п_р_и


In [263]:
dir(phrases)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_apply',
 '_load_specials',
 '_save_specials',
 '_smart_save',
 'add_vocab',
 'analyze_sentence',
 'common_terms',
 'corpus_word_count',
 'delimiter',
 'export_phrases',
 'learn_vocab',
 'load',
 'max_vocab_size',
 'min_count',
 'min_reduce',
 'progress_per',
 'save',
 'score_item',
 'scoring',
 'threshold',
 'vocab']

In [290]:
def tokenize(text):
    tokens = []
    tmp = ''
    for bigram in phrases[text]:
        if bigram[0] in punctuation + ' \n«»–':
            if not tmp in ' \n':
                tokens.append(tmp)
            tmp = bigram[-1]
        elif bigram[-1] in punctuation + ' \n«»–':
            if not tmp in ' \n':
                tokens.append(tmp + bigram[0])
            tmp=''
        else:
            tmp += bigram.replace('_', '')
    return tokens

t = text[:200]
print(tokenize(t))

['Annotation', '«Анна', 'Каренина', ' это', 'сложное', 'психологически', 'утонченное', ' остропроблемное', 'произведение', ' насыщенное', 'приметами', 'времени', ' Л', '.Н', ' Толстой', 'на', 'страницах', 'произведения', 'показывает', 'как', 'рушатся']


### Итог
Возможно, если получше написать обёртку, будет работать хорошо.